In [195]:
import pandas as pd
df = pd.read_csv("COVID19_data.csv", index_col="ID")
print(df.shape)

(2054, 12)


### Missing values in categorical variables

In [196]:
from collections import Counter

for var in df:
  if df[var].dtypes == object:
    print(var, Counter(df[var]))

SEX Counter({'MALE': 1227, 'FEMALE': 825, nan: 2})
EXITUS Counter({'NO': 1684, 'YES': 329, nan: 41})
DESTINATION Counter({nan: 1383, 'ADMISSION': 671})


### Zero values in numerical variables

In [197]:
for var in df:
  if df[var].dtypes != object:
    print(var, Counter(df[var])[0])

AGE 0
DAYS_HOSPITAL 21
DAYS_ICU 1947
TEMP 467
HEART_RATE 448
GLUCOSE 2035
SAT_O2 422
BLOOD_PRES_SYS 750
BLOOD_PRES_DIAS 750


### Relevant variables

In [198]:
# No values
df.pop("GLUCOSE")
df.pop("DESTINATION")

# Values to predict
days_hosp = df.pop("DAYS_HOSPITAL")
days_icu = df.pop("DAYS_ICU")
exitus_col = df.pop("EXITUS")

print(exitus_col)

ID
1       NO
2       NO
3       NO
4       NO
5       NO
        ..
2050    NO
2051    NO
2052    NO
2053    NO
2054    NO
Name: EXITUS, Length: 2054, dtype: object


In [199]:
print(df.isnull().any())

AGE                 True
SEX                 True
TEMP               False
HEART_RATE         False
SAT_O2             False
BLOOD_PRES_SYS     False
BLOOD_PRES_DIAS    False
dtype: bool


## Categorical Variables

In [200]:
# Mask for categorical variables
cat_mask = df.dtypes==object

cat_cols = df.columns[cat_mask].tolist()
print(cat_cols)

['SEX']


In [201]:
# Divide in categorical and numeric columns
df_cat = df[cat_cols]
df_num = df.drop(cat_cols, axis=1)

In [202]:
def imput_categorical(cat_var, imputer):
  return pd.DataFrame(
    imputer.fit_transform(cat_var),
    columns=cat_var.columns,
    index=cat_var.index
  )

In [203]:
from sklearn.impute import SimpleImputer

imp_cat = SimpleImputer(strategy='most_frequent')
df_cat = imput_categorical(df_cat, imp_cat)
df_exitus = imput_categorical(pd.DataFrame(exitus_col), imp_cat)

print(df_cat)
print(df_cat.isnull().any())

print(df_exitus)
print(df_exitus.isna().any())

         SEX
ID          
1     FEMALE
2     FEMALE
3       MALE
4       MALE
5       MALE
...      ...
2050  FEMALE
2051  FEMALE
2052  FEMALE
2053    MALE
2054    MALE

[2054 rows x 1 columns]
SEX    False
dtype: bool
     EXITUS
ID         
1        NO
2        NO
3        NO
4        NO
5        NO
...     ...
2050     NO
2051     NO
2052     NO
2053     NO
2054     NO

[2054 rows x 1 columns]
EXITUS    False
dtype: bool


In [204]:
def categorical_to_onehot(cat_var, encoder):
  return pd.DataFrame(
    encoder.fit_transform(cat_var),
    columns=encoder.get_feature_names_out(cat_var.columns.tolist()),
    index=cat_var.index
  )

In [205]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)

df_cat_one = categorical_to_onehot(df_cat, ohe)
df_exitus_one = categorical_to_onehot(df_exitus, ohe)
print(df_exitus_one)

      EXITUS_NO  EXITUS_YES
ID                         
1           1.0         0.0
2           1.0         0.0
3           1.0         0.0
4           1.0         0.0
5           1.0         0.0
...         ...         ...
2050        1.0         0.0
2051        1.0         0.0
2052        1.0         0.0
2053        1.0         0.0
2054        1.0         0.0

[2054 rows x 2 columns]


## Numerical Variables

In [206]:
df_num.isna().any()

AGE                 True
TEMP               False
HEART_RATE         False
SAT_O2             False
BLOOD_PRES_SYS     False
BLOOD_PRES_DIAS    False
dtype: bool

In [207]:
df_num['AGE'].fillna(0.0, inplace=True)
df_num.isna().any()

AGE                False
TEMP               False
HEART_RATE         False
SAT_O2             False
BLOOD_PRES_SYS     False
BLOOD_PRES_DIAS    False
dtype: bool

In [208]:
imp_num = SimpleImputer(missing_values=0.0, strategy='mean', fill_value=0.0)
df_num = pd.DataFrame(
  imp_num.fit_transform(df_num),
  columns=df_num.columns,
  index=df_num.index
)
print(df_num.isnull().any())
print(df_num)

AGE                False
TEMP               False
HEART_RATE         False
SAT_O2             False
BLOOD_PRES_SYS     False
BLOOD_PRES_DIAS    False
dtype: bool
             AGE       TEMP  HEART_RATE     SAT_O2  BLOOD_PRES_SYS  \
ID                                                                   
1      15.000000  37.000000   90.534247  92.000000      131.638037   
2      18.000000  37.300000  105.000000  97.000000      131.638037   
3      21.000000  38.500000  112.000000  95.000000       85.000000   
4      21.000000  39.200000  113.000000  97.000000      131.638037   
5      22.000000  36.300000   80.000000  92.000000      111.000000   
...          ...        ...         ...        ...             ...   
2050  189.000000  36.739445   90.534247  92.376838      131.638037   
2051   70.856585  36.500000   90.534247  92.376838      131.638037   
2052   70.856585  36.800000  190.000000  98.000000      131.638037   
2053   70.856585  36.739445  120.000000  93.000000      131.638037  

## Merge all the processed variables

In [214]:
df_processed = pd.merge(
  left=df_cat_one,
  right=df_num,
  on='ID'
)

print(df_processed)

      SEX_FEMALE  SEX_MALE         AGE       TEMP  HEART_RATE     SAT_O2  \
ID                                                                         
1            1.0       0.0   15.000000  37.000000   90.534247  92.000000   
2            1.0       0.0   18.000000  37.300000  105.000000  97.000000   
3            0.0       1.0   21.000000  38.500000  112.000000  95.000000   
4            0.0       1.0   21.000000  39.200000  113.000000  97.000000   
5            0.0       1.0   22.000000  36.300000   80.000000  92.000000   
...          ...       ...         ...        ...         ...        ...   
2050         1.0       0.0  189.000000  36.739445   90.534247  92.376838   
2051         1.0       0.0   70.856585  36.500000   90.534247  92.376838   
2052         1.0       0.0   70.856585  36.800000  190.000000  98.000000   
2053         0.0       1.0   70.856585  36.739445  120.000000  93.000000   
2054         0.0       1.0   70.856585  36.800000   90.534247  92.376838   

      BLOOD

## Feature Selection

In [211]:
from sklearn.feature_selection import SelectPercentile, mutual_info_classif

fs_perc_mi = SelectPercentile(mutual_info_classif, percentile=40)
fs_perc_mi.fit(df_processed, df_exitus['EXITUS'])
col_filter = fs_perc_mi.get_support()
df_perc_mi = df_processed.iloc[:, col_filter]

print(df_perc_mi)

             AGE     SAT_O2  BLOOD_PRES_SYS
ID                                         
1      15.000000  92.000000      131.638037
2      18.000000  97.000000      131.638037
3      21.000000  95.000000       85.000000
4      21.000000  97.000000      131.638037
5      22.000000  92.000000      111.000000
...          ...        ...             ...
2050  189.000000  92.376838      131.638037
2051   70.856585  92.376838      131.638037
2052   70.856585  98.000000      131.638037
2053   70.856585  93.000000      131.638037
2054   70.856585  92.376838      131.638037

[2054 rows x 3 columns]


### Feature Scaling

In [223]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
df_num_mms = mms.fit_transform(df_num)
print(df_num_mms)

[[0.         0.55072464 0.12156337 0.92134831 0.15962997 0.07808661]
 [0.01724138 0.5942029  0.14685315 0.97752809 0.15962997 0.07808661]
 [0.03448276 0.76811594 0.15909091 0.95505618 0.0984252  0.04316547]
 ...
 [0.32101486 0.52173913 0.29545455 0.98876404 0.15962997 0.07808661]
 [0.32101486 0.51296312 0.17307692 0.93258427 0.15962997 0.07808661]
 [0.32101486 0.52173913 0.12156337 0.92558245 0.15962997 0.07808661]]
